In [1]:
from os import environ
environ.setdefault("HF_HOME", "/data/hf_models/")

'/data/hf_models/'

In [2]:
import sys
sys.path.append('../../src/')

In [ ]:
from agent_design_pattern.agent import AgentMessage, BaseAgent
from agent_design_pattern.orchestration import ParallelAgent
from chain import CasualSingleTurnChain

In [4]:
class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [5]:
system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

chain = CasualSingleTurnChain(model="HuggingFaceTB/SmolLM3-3B", system_prompt=system_prompt, user_prompt_template=user_prompt)
chain2 = CasualSingleTurnChain("ibm-granite/granite-4.0-h-1b", system_prompt, user_prompt)
agent1 = Agent(chain, state_change_callback=state_callback)
agent2 = Agent(chain2, state_change_callback=state_callback)
parallel_agent = ParallelAgent([agent1, agent2], state_change_callback=state_callback)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]
The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [6]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query), temperature=1.5, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


query='' origin='ParallelAgent_3' response=None responses=[('Agent_1', '\n\n**"Fame as Frost, Love as Fire: A Bard\'s Elegy"**  \n\nNow fame’s tall star like frost doth cold-eyed freeze,  \nWith time, Love’s soul may with its light once might appease.  \nAnd yet if Time’s strong whip were but to burn my soul,  \nWould it not yet by fate’s fire in flames outdraw?  \n\nWhere shadows danced once in sweetest sunlight’s play,  \nThe flower we’ve once named Love has with Spring, with decay.  \nYet when a heart hath found the warmth of some pure flame—  \nIt fears not frost, no, ’tis death to stand unmoved in storm’s gaze.  \n\nAnd so shall I, my Love, like one star so rare,  \nThat rises at dusk on a world now worn gray:  \nTo bask in Time’s last ray like a tender flower  \nBut turn with him still toward Love and let fall—no!—not a star to fade, but forever whole!'), ('Agent_2', '\n\n**"Fame as Frost, Love as Fire: A Bard\'s Elegy"**  \n\nNow fame’s tall star like frost doth cold-eyed freeze